# Flood Impact Analysis for Somalia

### Description:
This Jupyter Notebook automates the analysis of flood impacts on various geospatial features in Somalia, including:
- **Buildings:** Identifies the number of buildings affected by floods.
- **Roads:** Calculates the total length of roads impacted.
- **Landcover:** Assesses agricultural and land use types within the flood zone.
- **Settlements:** Analyzes settlements within 500 meters of the flood extent.

The results are compiled into summary statistics for reporting and decision-making.

### Authors:
- **Ismail Khalif, SWALIM FAO Sonalia**
- **Abdillahi Osman Omar, ITC, University of Twente**  
  *Email: abdillahiosmanomar@student.utwente.nl*

### Date:
- **02/12/2024**

---

### How to Use:
1. Install the required software: Ensure you have ArcGIS Pro or ArcPy installed and configured.
2. Update the paths: Customize the workspace, geodatabase, and layer paths to match your environment.
3. Run the Notebook: Execute the cells step-by-step or run all cells at once to perform the analysis.
4. View Results: Check the output layers and summary table generated in the specified geodatabase.

### Requirements:
- **ArcGIS Pro (with ArcPy enabled)**
- **Python 3.x**
- **Jupyter Notebook**

---


## 01. Import Python libraries and initialize the environment

### 01a. Install Python libraries
```python
# !pip install arcpy
# !pip install os


### 01b. Import Python libraries

In [2]:
#import arcpy
#import os

## 02. Define Workspace and Geodatabase Paths

### 02a. Set up workspace and paths

In [ ]:
gdb_path = r".../OneDrive - Food and Agriculture Organization\Flood impact analysis-UNFAO-ND10822DW\SOM Flood impact analysis\SOM Flood impact analysis.gdb"
arcpy.env.workspace = gdb_path
arcpy.env.overwriteOutput = True

### 02b. Input Layers

In [ ]:
flood_extent_layer = "SOM_BeletWeyne_Floods_20_11_2024_v3"
buildings_layer = "c3d7_buildings_BELETEYNE"
roads_layer = "gis_osm_roads_free_1_ExportFeatures"
landcover_layer = "Som_Agrimask_Hiraan_Region_UTM_17102022"
settlements_layer = "harmonised_settlement_14112022"


### 02c. Define Output Paths

In [ ]:
flood_analysis_dataset = os.path.join(gdb_path, "Flood_Analysis")
if not arcpy.Exists(flood_analysis_dataset):
    spatial_ref = arcpy.Describe(flood_extent_layer).spatialReference
    arcpy.CreateFeatureDataset_management(gdb_path, "Flood_Analysis", spatial_ref)

clipped_buildings = os.path.join(flood_analysis_dataset, "Clipped_Buildings")
clipped_roads = os.path.join(flood_analysis_dataset, "Clipped_Roads")
clipped_landcover = os.path.join(flood_analysis_dataset, "Clipped_Landcover")
buffer_output = os.path.join(gdb_path, "FloodExtent_Buffer_500m")
intersect_output = os.path.join(gdb_path, "Settlements_Intersect_Buffer")
main_stats_table = os.path.join(gdb_path, "FloodStatisticsTable")
categorized_landcover_table = os.path.join(gdb_path, "CategorizedLandcoverTable")


## 03. Clip Layers to Flood Extent

### 03a. Clip Layers

In [ ]:
if not arcpy.Exists(clipped_buildings):
    arcpy.analysis.Clip(buildings_layer, flood_extent_layer, clipped_buildings)
    print("Clipped buildings to flood extent.")

if not arcpy.Exists(clipped_roads):
    arcpy.analysis.Clip(roads_layer, flood_extent_layer, clipped_roads)
    print("Clipped roads to flood extent.")

if not arcpy.Exists(clipped_landcover):
    arcpy.analysis.Clip(landcover_layer, flood_extent_layer, clipped_landcover)
    print("Clipped landcover to flood extent.")


## 04. Calculate General Statistics

### 04a. Calculate Statistics

In [ ]:
# Buildings count
building_count = int(arcpy.management.GetCount(clipped_buildings)[0])
print(f"Number of buildings in flood area: {building_count}")

# Roads length in kilometers
if "Road_Length_km" not in [field.name for field in arcpy.ListFields(clipped_roads)]:
    arcpy.management.AddField(clipped_roads, "Road_Length_km", "DOUBLE")
arcpy.management.CalculateGeometryAttributes(
    clipped_roads, [["Road_Length_km", "LENGTH_GEODESIC"]], length_unit="KILOMETERS"
)
road_length = sum(row[0] for row in arcpy.da.SearchCursor(clipped_roads, ["Road_Length_km"]))
print(f"Total road length in flood area: {road_length:.2f} km")

# Total flooded area in hectares
if "Area_Ha" not in [field.name for field in arcpy.ListFields(flood_extent_layer)]:
    arcpy.management.AddField(flood_extent_layer, "Area_Ha", "DOUBLE")
arcpy.management.CalculateGeometryAttributes(
    flood_extent_layer, [["Area_Ha", "AREA_GEODESIC"]], area_unit="HECTARES"
)
total_flooded_area = sum(row[0] for row in arcpy.da.SearchCursor(flood_extent_layer, ["Area_Ha"]))
print(f"Total flooded area: {total_flooded_area:.2f} hectares")


## 05. Categorize Agricultural Land Statistics

### 05a. Classify Landcover

In [ ]:
# Define categories of interest
selected_classes = {
    "HI": "Herbaceous Crop Irrigated",
    "SI": "Shrub Irrigated Crop",
    "HR": "Herbaceous Crop Rainfed",
    "TI": "Tree Crop Irrigated"
}

# Add a new "Category" field if it doesn't exist
if "Category" not in [field.name for field in arcpy.ListFields(clipped_landcover)]:
    arcpy.management.AddField(clipped_landcover, "Category", "TEXT")

# Update "Category" based on the selected landcover classes
with arcpy.da.UpdateCursor(clipped_landcover, ["LCCS", "Category"]) as cursor:
    for row in cursor:
        if row[0] in selected_classes:  # Check if the LCCS value is in the selected classes
            row[1] = selected_classes[row[0]]
        else:
            row[1] = "Other"
        cursor.updateRow(row)
print("Categorized agricultural land statistics.")


### 05b. Save Categorized Statistics

In [ ]:
if "Area_Ha" not in [field.name for field in arcpy.ListFields(clipped_landcover)]:
    arcpy.management.AddField(clipped_landcover, "Area_Ha", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        clipped_landcover, [["Area_Ha", "AREA_GEODESIC"]], area_unit="HECTARES"
    )

if not arcpy.Exists(categorized_landcover_table):
    arcpy.analysis.Statistics(
        clipped_landcover,
        categorized_landcover_table,
        [["Area_Ha", "SUM"]],
        "Category"
    )
    print("Categorized landcover statistics saved.")


## 06. Analyze Settlements Within 500m Buffer

### 06a. Buffer and Intersect

In [ ]:
if not arcpy.Exists(buffer_output):
    arcpy.analysis.Buffer(flood_extent_layer, buffer_output, "500 Meters", dissolve_option="ALL")
    print("Created buffer around flood extent.")

if not arcpy.Exists(intersect_output):
    arcpy.analysis.Intersect([buffer_output, settlements_layer], intersect_output)
    print("Intersected settlements with flood buffer.")

settlement_count = int(arcpy.management.GetCount(intersect_output)[0])
print(f"Number of settlements within 500 meters of flooded area: {settlement_count}")


## 07. Compile Statistics Table

### 07a. Compile General Statistics

In [ ]:
if not arcpy.Exists(main_stats_table):
    arcpy.management.CreateTable(gdb_path, "FloodStatisticsTable")
    arcpy.management.AddField(main_stats_table, "Description", "TEXT")
    arcpy.management.AddField(main_stats_table, "Result", "TEXT")

# Insert new statistics
with arcpy.da.InsertCursor(main_stats_table, ["Description", "Result"]) as cursor:
    cursor.insertRow(("Total Flooded Area", f"{total_flooded_area:.2f} Ha"))
    cursor.insertRow(("Total Settlements within 500m", str(settlement_count)))
    cursor.insertRow(("Total Buildings", str(building_count)))
    cursor.insertRow(("Total Road Length", f"{road_length:.2f} km"))
    cursor.insertRow(("Total Agricultural Flooded Area", f"{total_agriculture_area:.2f} Ha"))

print("Flood statistics compiled into the main table.")


### **Conclusion**

This analysis assessed the flood impact in BeletWeyne, Somalia, by clipping spatial layers (buildings, roads, land cover) to the flood extent and calculating key statistics, including buildings, road length, and flooded area. Agricultural land categorization quantified crop loss, and settlements within the flood buffer highlighted areas of vulnerability.

The results provide valuable insights for emergency planning and resource allocation. The methodology, leveraging ArcGIS tools, can be applied to similar flood events, with potential for further integration of socioeconomic data for enhanced vulnerability assessment and mitigation strategies.

If you have any questions, please contact the authors.

### Authors:
- **Ismail Khalif, SWALIM FAO Sonalia**
- **Abdillahi Osman Omar, ITC, University of Twente**  
  *Email: abdillahiosmanomar@student.utwente.nl*
